In [ ]:
import pandas as pd
import rdkit.Chem as Chem
from rdkit.rdBase import BlockLogs
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from rdkit.Chem.MolStandardize import rdMolStandardize # type: ignore
from rdkit.Chem.Descriptors import CalcMolDescriptors
from sklearn.model_selection import GroupShuffleSplit, KFold
import numpy as np
from utils import standardize, get_scaffold

from torch.utils.data import Dataset, DataLoader
import random
from chemprop.featurizers.molgraph.reaction import CondensedGraphOfReactionFeaturizer
from chemprop.data.datapoints import ReactionDatapoint
from chemprop.data.datasets import Datum


RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [2]:
def mol_to_inchi(mol):
    with BlockLogs():
        return Chem.MolToInchi(mol)

df = pd.read_csv("./GSK_HepG2.csv")
df = df.iloc[:, 1:]
df.columns = ['smiles', 'per_inhibition']


# standardize and convert to inchi
df['mol'] = df['smiles'].map(standardize)
df = df.dropna(subset=['mol'])
df['inchi'] = df['mol'].map(mol_to_inchi)
df = df.groupby(["inchi"]).filter(lambda x: len(x) == 1).reset_index(drop=True)

clusters, _ = pd.factorize(
    df['mol']
        .map(Chem.MolToSmiles) # type: ignore
        .map(get_scaffold)
)
clusters = pd.Series(clusters)


df = df.drop(['smiles', 'inchi'], axis=1)

In [5]:
splitter = GroupShuffleSplit(n_splits=1, random_state=RANDOM_SEED)
train_idxs, val_test_idxs = next(splitter.split(df, groups=clusters))
df_train = df.loc[train_idxs].reset_index(drop=True)
df_val_test = df.loc[val_test_idxs].reset_index(drop=True)
clusters_val_test = clusters.iloc[val_test_idxs].reset_index(drop=True)


splitter = GroupShuffleSplit(n_splits=1, random_state=RANDOM_SEED, test_size=0.5)
val_idxs, test_idxs = next(splitter.split(df_val_test, groups=clusters_val_test))
df_val = df_val_test.loc[val_idxs].reset_index(drop=True)
df_test = df_val_test.loc[test_idxs].reset_index(drop=True)

In [6]:
from sklearn.preprocessing import StandardScaler

class PairDataset(Dataset):
    def __init__(self, data):
        self.data = data  # raw data
        self.featurizer = CondensedGraphOfReactionFeaturizer()
        self.scaler = None
        self.pairs = []
        self.deltas: np.ndarray | None = None
        self.update_pairs_and_deltas()  # list of (i, j, label)
        

    def update_pairs(self):
        idxs = np.arange(len(self.data))
        shuffled_idxs = np.arange(len(self.data))
        np.random.shuffle(shuffled_idxs)
        
        pairs = list(zip(idxs, shuffled_idxs))
        swapped = list(zip(shuffled_idxs, idxs))
        same = list(zip(idxs, idxs))
        self.pairs = pairs + swapped[: len(swapped) // 2] + same[: len(same) // 4]


    def update_deltas(self):
        left_idxs, right_idxs = list(zip(*self.pairs))
        left_idxs = list(left_idxs)
        right_idxs = list(right_idxs)

        left = self.data.iloc[left_idxs, : ].reset_index(drop=True)
        right = self.data.iloc[right_idxs, : ].reset_index(drop=True)
        deltas = (left['per_inhibition'].astype(float) - right['per_inhibition'].astype(float))
        self.deltas = deltas.to_numpy().reshape(-1, 1)


    def update_pairs_and_deltas(self):
        self.update_pairs()
        self.update_deltas()


    def normalize_targets(self, scaler=None):
        assert self.deltas is not None
        if scaler is None:
            scaler = StandardScaler().fit(self.deltas)

        self.deltas = scaler.transform(self.deltas)
        return scaler
        
    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        i, j = self.pairs[idx]
        left = self.data['mol'][i]
        right = self.data['mol'][j]
        assert self.deltas is not None
        delta = self.deltas[idx]

        mg = self.featurizer((left, right), None, None)
        rxn_dp = ReactionDatapoint(left, right, delta)
        return Datum(mg, None, None, delta, rxn_dp.weight, rxn_dp.lt_mask, rxn_dp.gt_mask)

In [ ]:
from chemprop.data.dataloader import build_dataloader

train_ds = PairDataset(df_train)
scaler = train_ds.normalize_targets()

val_ds = PairDataset(df_val)
scaler = val_ds.normalize_targets(scaler)

In [8]:
import lightning as L
class MyDataModule(L.LightningDataModule):
    def __init__(self, train_dataset: PairDataset, val_dataset: PairDataset):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset

    def setup(self, stage=None):
        pass

    def train_dataloader(self):
        self.train_dataset.update_pairs_and_deltas()
        self.train_dataset.normalize_targets(scaler)
        return build_dataloader(
            self.train_dataset,   # type: ignore
            batch_size=512,
            num_workers=4,
        )
    
    def val_dataloader(self):
        return build_dataloader(
            self.val_dataset,   # type: ignore
            batch_size=512,
            num_workers=4,
            shuffle=False
        )

In [18]:
from chemprop import data, featurizers, models, nn

fdims = train_ds.featurizer.shape # the dimensions of the featurizer, given as (atom_dims, bond_dims).
mp = nn.BondMessagePassing(*fdims)
agg = nn.NormAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(output_transform=output_transform)
batch_norm = True
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE(), nn.metrics.R2Score()]
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn.max_lr = 0.01

In [ ]:
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="mm")
trainer = L.Trainer(
    logger=logger,
    enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=80,  # number of epochs to train for
    reload_dataloaders_every_n_epochs=2
)

trainer.fit(mpnn, datamodule=MyDataModule(train_ds, val_ds))

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/rahul/delta/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 252 K  | train
1 | agg             | NormAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 60

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

/home/rahul/delta/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/rahul/delta/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
